In [ ]:
import numpy as np,sys
from sklearn.utils import shuffle
from tensorflow.keras.datasets.mnist import load_data
import matplotlib.pyplot as plt
import matplotlib
np.random.seed(678)

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-1 * x))
def d_sigmoid(x):
    return log(x) * ( 1 - log(x))

In [ ]:
def tanh(x):
    return np.tanh(x)
def d_tanh(x):
    return 1 - np.tanh(x) ** 2 

In [ ]:
def ReLu(x):
    return max(0, x)
def d_ReLu(x):
    return 1 if x > 0 else 0

In [ ]:
def elu(matrix):
    mask = (matrix<=0) * 1.0
    less_zero = matrix * mask
    safe =  (matrix>0) * 1.0
    greater_zero = matrix * safe
    final = 3.0 * (np.exp(less_zero) - 1) * less_zero
    return greater_zero + final
def d_elu(matrix):
    safe = (matrix>0) * 1.0
    mask2 = (matrix<=0) * 1.0
    temp = matrix * mask2
    final = (3.0 * np.exp(temp))*mask2
    return (matrix * safe) + final

In [ ]:
mnist = load_data
train = mnist.test
images, labels = train.images, train.labels
only_zero_index,only_one_index = np.where(labels==0)[0],np.where(labels==1)[0]
only_zero_image,only_zero_label = images[[only_zero_index]],np.expand_dims(labels[[only_zero_index]],axis=1)
only_one_image,only_one_label   = images[[only_one_index]],np.expand_dims(labels[[only_one_index]],axis=1)

images = np.vstack((only_zero_image,only_one_image))
labels = np.vstack((only_zero_label,only_one_label))
images,label = shuffle(images,labels)

test_image_num,training_image_num = 20,100
testing_images, testing_lables =images[:test_image_num,:],label[:test_image_num,:]
training_images,training_lables =images[test_image_num:test_image_num+training_image_num,:],label[test_image_num:test_image_num+training_image_num,:]


In [ ]:
w1 = np.random.randn(784,256) * 0.2
w2 =np.random.randn(256,128) * 0.2
w3 =np.random.randn(128,1) * 0.2

w1_MBDescent,w2_MBDescent,w3_MBDescent=w1,w2,w3
w1_RSMprop,w2_RSMprop,w3_RSMprop=w1,w2,w3
w1_adam,w2_adam,w3_adam=w1,w2,w3

In [ ]:
num_epoch = 500
total_cost = 0
learn_rate = 0.0003
cost_array =[]

In [ ]:
#MOMENTUM
v1,v2,v3 = 0,0,0
alpha = 0.001
total_cost = 0
cost_temp_array = []
for iter in range(num_epoch):
    for image_index in range(len(training_images)):
        
        current_image = np.expand_dims(training_images[image_index],axis=0)
        current_image_label = np.expand_dims(training_lables[image_index],axis=1)

        l1 = current_image.dot(w1_m)
        l1A = elu(l1)

        l2 = l1A.dot(w2_m)
        l2A = tanh(l2)       

        l3 = l2A.dot(w3_m)
        l3A = log(l3)   

        cost = np.square(l3A - current_image_label).sum() * 0.5
        total_cost = total_cost + cost

        grad_3_part_1 =l3A - current_image_label
        grad_3_part_2 =d_log(l3)
        grad_3_part_3 =l2A
        grad_3 =grad_3_part_3.T.dot(grad_3_part_1 * grad_3_part_2)    

        grad_2_part_1 =(grad_3_part_1 * grad_3_part_2).dot(w3_m.T)
        grad_2_part_2 =d_tanh(l2)
        grad_2_part_3 =l1A
        grad_2 =grad_2_part_3.T.dot(grad_2_part_1 * grad_2_part_2)

        grad_1_part_1 =(grad_2_part_1 * grad_2_part_2).dot(w2_m.T)
        grad_1_part_2 =d_elu(l1)
        grad_1_part_3 =current_image
        grad_1 =grad_1_part_3.T.dot(grad_1_part_1 *grad_1_part_2)

        v3 =v3 * alpha + learn_rate * grad_3
        v2 =v2 * alpha + learn_rate * grad_2
        v1 =v1 * alpha + learn_rate * grad_1

        w3_m =w3_m - v3
        w2_m =w2_m - v2
        w1_m =w1_m - v1
    if iter %10 == 0 :
        print("b. Momentum current Iter: ", iter, " Total Cost: ", total_cost)
    cost_temp_array.append(total_cost)
    total_cost = 0
cost_array.append(cost_temp_array)

In [ ]:
#RMSProp
RMSprop_1,RMSprop_2,RMSprop_3 = 0,0,0
RMSprop_v,RMSprop_e= 0.9,0.00000001
total_cost = 0
cost_temp_array = []
print('-------------------------')
for iter in range(num_epoch):
    for image_index in range(len(training_images)):
        
        current_image = np.expand_dims(training_images[image_index],axis=0)
        current_image_label = np.expand_dims(training_lables[image_index],axis=1)

        l1 = current_image.dot(w1_RSMprop)
        l1A = elu(l1)

        l2 = l1A.dot(w2_RSMprop)
        l2A = tanh(l2)       

        l3 = l2A.dot(w3_RSMprop)
        l3A = log(l3)   

        cost = np.square(l3A - current_image_label).sum() * 0.5
        total_cost = total_cost + cost

        grad_3_part_1 =l3A - current_image_label
        grad_3_part_2 =d_log(l3)
        grad_3_part_3 =l2A
        grad_3 =grad_3_part_3.T.dot(grad_3_part_1 * grad_3_part_2)    

        grad_2_part_1 =(grad_3_part_1 * grad_3_part_2).dot(w3_RSMprop.T)
        grad_2_part_2 =d_tanh(l2)
        grad_2_part_3 =l1A
        grad_2 =grad_2_part_3.T.dot(grad_2_part_1 * grad_2_part_2)

        grad_1_part_1 =(grad_2_part_1 * grad_2_part_2).dot(w2_RSMprop.T)
        grad_1_part_2 =d_elu(l1)
        grad_1_part_3 =current_image
        grad_1 =grad_1_part_3.T.dot(grad_1_part_1 *grad_1_part_2)

        RMSprop_3 =RMSprop_v*RMSprop_3 + (1- RMSprop_v)*grad_3**2
        RMSprop_2 =RMSprop_v*RMSprop_2 + (1- RMSprop_v)*grad_2**2
        RMSprop_1 =RMSprop_v*RMSprop_1 + (1- RMSprop_v)*grad_1**2

        w3_RSMprop =w3_RSMprop - (learn_rate/np.sqrt(RMSprop_3 + RMSprop_e)) * grad_3
        w2_RSMprop =w2_RSMprop - (learn_rate/np.sqrt(RMSprop_2 + RMSprop_e)) * grad_2
        w1_RSMprop =w1_RSMprop - (learn_rate/np.sqrt(RMSprop_1 + RMSprop_e)) * grad_1
    if iter %10 == 0 :
        print("f. RMSprop current Iter: ", iter, " Total Cost: ", total_cost)
    cost_temp_array.append(total_cost)
    total_cost = 0
cost_array.append(cost_temp_array)

In [ ]:
#Adam
Adam_m_1,Adam_m_2,Adam_m_3 = 0,0,0
Adam_v_1,Adam_v_2,Adam_v_3 = 0,0,0
Adam_Beta_1,Adam_Beta_2 = 0.9,0.999
Adam_e = 0.00000001
total_cost = 0
cost_temp_array = []
print('-------------------------')
for iter in range(num_epoch):
    for image_index in range(len(training_images)):
        
        current_image = np.expand_dims(training_images[image_index],axis=0)
        current_image_label = np.expand_dims(training_lables[image_index],axis=1)

        l1 = current_image.dot(w1_adam)
        l1A = elu(l1)

        l2 = l1A.dot(w2_adam)
        l2A = tanh(l2)       

        l3 = l2A.dot(w3_adam)
        l3A = log(l3)   

        cost = np.square(l3A - current_image_label).sum() * 0.5
        total_cost = total_cost + cost

        grad_3_part_1 =l3A - current_image_label
        grad_3_part_2 =d_log(l3)
        grad_3_part_3 =l2A
        grad_3 =grad_3_part_3.T.dot(grad_3_part_1 * grad_3_part_2)    

        grad_2_part_1 =(grad_3_part_1 * grad_3_part_2).dot(w3_adam.T)
        grad_2_part_2 =d_tanh(l2)
        grad_2_part_3 =l1A
        grad_2 =grad_2_part_3.T.dot(grad_2_part_1 * grad_2_part_2)

        grad_1_part_1 =(grad_2_part_1 * grad_2_part_2).dot(w2_adam.T)
        grad_1_part_2 =d_elu(l1)
        grad_1_part_3 =current_image
        grad_1 =grad_1_part_3.T.dot(grad_1_part_1 *grad_1_part_2)

        Adam_m_3 =Adam_Beta_1*Adam_m_3+( 1-Adam_Beta_1 )*grad_3
        Adam_m_2 =Adam_Beta_1*Adam_m_2+( 1-Adam_Beta_1 )*grad_2
        Adam_m_1 =Adam_Beta_1*Adam_m_1+( 1-Adam_Beta_1 )*grad_1

        Adam_v_3 =Adam_Beta_2*Adam_v_3+( 1-Adam_Beta_2 )*grad_3 **2 
        Adam_v_2 =Adam_Beta_2*Adam_v_2+( 1-Adam_Beta_2 )*grad_2 **2 
        Adam_v_1 =Adam_Beta_2*Adam_v_1+( 1-Adam_Beta_2 )*grad_1 **2 
        
        Adam_m_3_hat = Adam_m_3/(1-Adam_Beta_1)
        Adam_m_2_hat = Adam_m_2/(1-Adam_Beta_1)
        Adam_m_1_hat = Adam_m_1/(1-Adam_Beta_1)
        
        Adam_v_3_hat = Adam_v_3/(1-Adam_Beta_2)
        Adam_v_2_hat = Adam_v_2/(1-Adam_Beta_2)
        Adam_v_1_hat = Adam_v_1/(1-Adam_Beta_2)
        
        w3_adam = w3_adam - (learn_rate/(np.sqrt(Adam_v_3_hat) + Adam_e)) * Adam_m_3_hat
        w2_adam = w2_adam - (learn_rate/(np.sqrt(Adam_v_2_hat) + Adam_e)) * Adam_m_2_hat
        w1_adam = w1_adam - (learn_rate/(np.sqrt(Adam_v_1_hat) + Adam_e)) * Adam_m_1_hat
        
    if iter %10 == 0 :
        print("g. Adam current Iter: ", iter, " Total Cost: ", total_cost)
    cost_temp_array.append(total_cost)
    total_cost = 0
cost_array.append(cost_temp_array)